In [2]:
%pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 4.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession

In [4]:
#spark = SparkSession.builder \
    #.appName("Read MovieLens from HDFS") \
    #.getOrCreate()

#spark = SparkSession.builder \
    #.appName("ALS Model") \
    #.config("spark.driver.memory", "4g") \
    #.config("spark.executor.memory", "4g") \
   # .getOrCreate()

#.config("spark.hadoop.fs.defaultFS", "hdfs://bigdata-node:8088") \
spark = SparkSession.builder \
    .appName("MovieLensAnalysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .config("spark.sql.shuffle.partitions", "20") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/02 09:45:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
ratings_df = spark.read.option("header", True).csv("hdfs:///data/rating.csv")

In [6]:
ratings_df.show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



In [7]:
movies_df = spark.read.option("header", True).csv("hdfs:///data/movie.csv")

In [8]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [9]:
df_data = ratings_df.join(
    movies_df.select("movieId", "title"),
    on="movieId",
    how="left"
)

In [10]:
df_data.show(5)

+-------+------+------+-------------------+--------------------+
|movieId|userId|rating|          timestamp|               title|
+-------+------+------+-------------------+--------------------+
|      2|     1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|
|     29|     1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|
|     32|     1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|
|     47|     1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|
|     50|     1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|
+-------+------+------+-------------------+--------------------+
only showing top 5 rows



In [11]:
df_data = df_data.withColumnsRenamed({
    'userId': 'user',
    'movieId': 'movie',
})

In [12]:
df_data.show(5)

+-----+----+------+-------------------+--------------------+
|movie|user|rating|          timestamp|               title|
+-----+----+------+-------------------+--------------------+
|    2|   1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|
|   29|   1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|
|   32|   1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|
|   47|   1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|
|   50|   1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|
+-----+----+------+-------------------+--------------------+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import col

df_data = df_data \
    .withColumn("user", col("user").cast("int")) \
    .withColumn("movie", col("movie").cast("int")) \
    .withColumn("rating", col("rating").cast("float")) \
    .withColumn("timestamp", col("timestamp").cast("int")) \
    .withColumn("title", col("title").cast("string"))

In [14]:
df_data_train, df_data_test = df_data.randomSplit([0.8, 0.2], 42)

In [15]:
# import ALS model from spark
from pyspark.ml.recommendation import ALS

# initialise the model 
als = ALS(maxIter=10, regParam=0.1, rank=10, userCol="user", itemCol="movie", 
          ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

# train model 
als_model = als.fit(df_data_train)

25/05/02 09:46:25 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
                                                                                

In [1]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = als_model.transform(df_data_test)  # ou train, selon ce que tu veux
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print(f"RMSE = {rmse}")

ModuleNotFoundError: No module named 'numpy'

In [16]:
best_model = als_model.bestModel

AttributeError: 'ALSModel' object has no attribute 'bestModel'

In [22]:
dfs_preds = als_model.transform(df_data_test)

In [23]:
dfs_preds.show(5)

[Stage 114:==================================================>    (11 + 1) / 12]

+-----+----+------+---------+--------------------+----------+
|movie|user|rating|timestamp|               title|prediction|
+-----+----+------+---------+--------------------+----------+
|   47|  95|   3.0|     NULL|Seven (a.k.a. Se7...|  3.031183|
|  262|  95|   3.0|     NULL|Little Princess, ...| 3.3890905|
|  446|  95|   3.0|     NULL|Farewell My Concu...|  3.615186|
|  592|  95|   3.0|     NULL|       Batman (1989)| 2.4750078|
|  708|  95|   4.0|     NULL|Truth About Cats ...|  3.071957|
+-----+----+------+---------+--------------------+----------+
only showing top 5 rows

